## 各エンドポイントの説明

In [2]:
#新たに作成したモジュールなどをimport
import sys
import json
import requests
import os
import pandas as pd
sys.path.append("../src")
from utils import dataframe_to_dict

host_name = "http://localhost:8081"

## データベースをリセットする

In [13]:
url = host_name + "/reset_db"
res = requests.get(url)

print(res.content)

b'["success","success","false"]'


## 使用するデータを取得

In [7]:
# URL to the Titanic dataset
url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
# Read data from the URL
raw_data = pd.read_csv(url)

NameError: name 'pd' is not defined

In [207]:
raw_data[0:2]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


## データフレームを辞書型に変換する

空白があるデータなどは除外する

In [208]:
success_result_dict = dataframe_to_dict(raw_data[0:2])
print('~~~~成功例~~~~')
print(success_result_dict)

#['Survived', 'Pclass', 'Sex', 'Age', 'Fare']の項目が揃っていないデータはエラーを返す
print('~~~~失敗例~~~~~')
false_result_dict = dataframe_to_dict(raw_data[0:2]['Name'])
print(false_result_dict)

~~~~成功例~~~~
[{'Survived': 0, 'Pclass': 3, 'Sex': 'male', 'Age': 22.0, 'Fare': 7.25}, {'Survived': 1, 'Pclass': 1, 'Sex': 'female', 'Age': 38.0, 'Fare': 71.2833}]
~~~~失敗例~~~~~
"None of [Index(['Survived', 'Pclass', 'Sex', 'Age', 'Fare'], dtype='object')] are in the [index]"
正しいデータ形式ではありません。


## データベースにデータの登録を行う

In [1]:
#データを辞書型のリストに変換する
result_dict = dataframe_to_dict(raw_data[0:1000],mode='sex_to_num')

#データをデータベースに登録する
url = host_name + "/upload_data"
res = requests.post(url, json=result_dict)

print(res.content)

NameError: name 'dataframe_to_dict' is not defined

## データベースからデータを取得する

In [8]:
#DBからデータを取得する
url = host_name + "/get_data"
data_index={'start_index':1,'end_index':5}#取得するindexの範囲を指定する
res = requests.post(url, json=data_index)

#resはバイナリデータで返される, decodeで文字列に変換
#その後、{}で囲まれた部分を、json.loads()で辞書型データに変換
data_list = json.loads(res.content.decode('utf-8'))
df = pd.DataFrame(data_list)
df

,data_id,survived,sex,fare,pclass,upload_date,age
0,1,1,0,8.5167,3,2024-05-19T18:53:54,0
1,2,1,0,14.5000,2,2024-05-19T18:53:54,0
2,3,1,1,19.2583,3,2024-05-19T18:53:54,0
3,4,1,1,19.2583,3,2024-05-19T18:53:54,0


## 新しいモデルの作成

指定したindexの範囲のデータを利用してトレーニングを行う

In [212]:
data = {'start_index':0, 'end_index':150, 'my_model_name':'new_test'}#トレーニングに使うindexの範囲を指定する
url = host_name + "/train_new_model"
res = requests.post(url, json=data)

data_list = json.loads(res.content.decode('utf-8'))
print(data_list)

{'transaction_result': 'success'}


## データベースからモデルの情報を取得する

model_id に -1を指定するとすべてのモデルデータが取得できる

In [213]:
data = {'version_id_1' : -1}
url = host_name + "/get_model"
res = requests.post(url, json=data)

data_list = json.loads(res.content.decode('utf-8'))
df = pd.DataFrame(data_list[0])
df

,pclass_coef,model_version_id,age_coef,training_iteration,my_model_name,sex_coef,training_date,fare_coef
0,-1.01682,1,-0.040715,40,new_test,2.34646,2024-05-19T18:04:10,-0.008878


## モデルを評価する

version_id -> ２つのidを指定して、modelの比較ができる。idの指定は１つでも可能。

start(end)_index -> 指定した範囲のデータに対しての推論結果などを表示する

In [220]:
data = {'version_id_1' : 1,'version_id_2' : 2,'start_index':0, 'end_index':150}
url = host_name + "/evaluate_model"
res = requests.post(url, json=data)